# CH-05 AI 技術指標回測系統

## 5-2 強大的回測工具：backtesting.py

### 1️⃣ 安裝及匯入套件

In [1]:
!pip install openai
!pip install yfinance
!pip install backtesting
!pip install bokeh # 繪圖套件

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.1/192.1 kB 6.7 MB/s eta 0:00:00


In [2]:
from  openai import OpenAI, OpenAIError # 串接 OpenAI API
import yfinance as yf
import pandas as pd # 資料處理套件
import datetime as dt # 時間套件
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

/usr/local/lib/python3.12/dist-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


### 2️⃣ 取得股價資料

In [3]:
# 輸入股票代號
stock_id = "2330.tw"
# 抓取 5 年資料
#df = yf.download(stock_id, period="5y")
df = yf.Ticker(stock_id).history(period="5y")
# 計算指標
df['ma1'] = df['Close'].rolling(window=5).mean()
df['ma2'] = df['Close'].rolling(window=10).mean()
df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,ma1,ma2
Date,,,,,,,,,
2020-10-19 00:00:00+08:00,412.103371,416.197113,410.283930,416.197113,34470906,0.0,0.0,NaN,NaN
2020-10-20 00:00:00+08:00,414.377801,415.742382,410.284058,410.284058,25205638,0.0,0.0,NaN,NaN
2020-10-21 00:00:00+08:00,412.558193,414.832494,410.283892,412.103333,28058921,0.0,0.0,NaN,NaN
2020-10-22 00:00:00+08:00,409.374249,413.922852,408.919389,413.922852,23438547,0.0,0.0,NaN,NaN
2020-10-23 00:00:00+08:00,416.652018,416.652018,410.738835,411.193695,18777385,0.0,0.0,412.74021,NaN


### 3️⃣ 定義回測策略

In [4]:
class CrossStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    if crossover(self.data.ma1, self.data.ma2):
      self.buy(size=1)
    elif crossover(self.data.ma2, self.data.ma1):
      self.sell(size=1)

### 4️⃣ 回測結果

In [5]:
backtest = Backtest(df,
        CrossStrategy,
        cash=100000,
        commission=0.004,
        margin=1,
        hedging=False,
        trade_on_close=False,
        exclusive_orders=False,
        )
stats = backtest.run()

# 印出回測績效
print(stats)

# 查看詳細的交易紀錄
stats["_trades"].head()

Backtest.run:   0%|          | 0/1214 [00:00<?, ?bar/s]

Start                     2020-10-19 00:00...
End                       2025-10-17 00:00...
Duration                   1824 days 00:00:00
Exposure Time [%]                    67.98354
Equity Final [$]                 101384.68469
Equity Peak [$]                  101454.68469
Commissions [$]                     327.01534
Return [%]                            1.38468
Buy & Hold Return [%]               248.39261
Return (Ann.) [%]                     0.28563
Volatility (Ann.) [%]                 0.32056
CAGR [%]                              0.19017
Sharpe Ratio                          0.89104
Sortino Ratio                         1.37038
Calmar Ratio                          0.52347
Alpha [%]                            -0.61501
Beta                                  0.00805
Max. Drawdown [%]                    -0.54565
Avg. Drawdown [%]                    -0.08631
Max. Drawdown Duration     1120 days 00:00:00
Avg. Drawdown Duration       68 days 00:00:00
# Trades                          

/tmp/ipython-input-4280171815.py:10: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  stats = backtest.run()


,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,SL,TP,PnL,Commission,ReturnPct,EntryTime,ExitTime,Duration,Tag
0,1,15,31,416.652012,445.308232,None,None,25.208378,3.447841,0.060502,2020-11-09 00:00:00+08:00,2020-12-01 00:00:00+08:00,22 days,None
1,1,34,43,453.495674,470.805023,None,None,13.612146,3.697203,0.030016,2020-12-04 00:00:00+08:00,2020-12-17 00:00:00+08:00,13 days,None
2,1,47,73,464.405701,565.879929,None,None,97.353086,4.121143,0.209629,2020-12-23 00:00:00+08:00,2021-01-29 00:00:00+08:00,37 days,None
3,1,79,86,606.104126,558.566649,None,None,-52.196160,4.658683,-0.086117,2021-02-17 00:00:00+08:00,2021-02-26 00:00:00+08:00,9 days,None
4,1,98,102,561.771890,548.002868,None,None,-18.208122,4.439099,-0.032412,2021-03-17 00:00:00+08:00,2021-03-23 00:00:00+08:00,6 days,None


### 5️⃣ 回測繪圖

In [6]:
backtest.plot(plot_equity=True,
       plot_return=False,
       plot_pl=True,
       plot_volume=True,
       plot_drawdown=False,
       superimpose=True)

/usr/local/lib/python3.12/dist-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


GridPlot(id='p1329', ...)

### 6️⃣ 設定停利、停損點

In [7]:
class CrossStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    if crossover(self.data.ma1, self.data.ma2):
        # 買入時設置停損與停利價格
        self.buy(size=1,
            sl=self.data.Close[-1] * 0.90,
            tp=self.data.Close[-1] * 1.10)
    elif crossover(self.data.ma2, self.data.ma1):
        # 賣出時時設置停損與停利價格
        self.sell(size=1,
             sl=self.data.Close[-1] * 1.10,
             tp=self.data.Close[-1] * 0.90)

backtest = Backtest(df,
        CrossStrategy,
        cash=100000,
        commission=0.004,
        margin=1,
        hedging=False,
        trade_on_close=False,
        exclusive_orders=False,
        )
stats = backtest.run()
print(stats)

Backtest.run:   0%|          | 0/1214 [00:00<?, ?bar/s]

Start                     2020-10-19 00:00...
End                       2025-10-17 00:00...
Duration                   1824 days 00:00:00
Exposure Time [%]                    47.57202
Equity Final [$]                  99641.39915
Equity Peak [$]                  100050.74382
Commissions [$]                      360.4533
Return [%]                            -0.3586
Buy & Hold Return [%]               248.39261
Return (Ann.) [%]                    -0.07448
Volatility (Ann.) [%]                 0.12929
CAGR [%]                             -0.04962
Sharpe Ratio                         -0.57608
Sortino Ratio                        -0.79977
Calmar Ratio                         -0.18205
Alpha [%]                            -0.44687
Beta                                  0.00036
Max. Drawdown [%]                    -0.40914
Avg. Drawdown [%]                    -0.14392
Max. Drawdown Duration     1722 days 00:00:00
Avg. Drawdown Duration      597 days 00:00:00
# Trades                          

## 5-3 讓 AI 產生回測策略

### 7️⃣ 輸入 OpenAI API KEY

In [8]:
from google.colab import userdata

client = OpenAI(
    api_key=userdata.get('GEMINI_API_KEY'),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

### 8️⃣ 創建 Gemini-2.5-Flash 模型函式

In [9]:
# Gemini 2.5 Flash 模型
def get_reply(messages):
  try:
    response = client.chat.completions.create(
        model="gemini-2.5-flash",
        n=1,
        messages=messages)
    reply = response.choices[0].message.content
  except OpenAIError as err:
    reply = f"發生 {err.type} 錯誤\n{err.message}"
  return reply

# 設定 AI 角色, 使其依據使用者需求進行 df 處理
def ai_helper(df, user_msg):

  msg = [{
    "role":
    "system",
    "content":
    f"As a professional code generation robot, \
      I require your assistance in generating Python code \
      based on specific user requirements. To proceed, \
      I will provide you with a dataframe (df) that follows the \
      format {df.columns}. Your task is to carefully analyze the \
      user's requirements and generate the Python code \
      accordingly.Please note that your response should solely \
      consist of the code itself, \
      and no additional information should be included."
  }, {
    "role":
    "user",
    "content":
    f"The user requirement:{user_msg} \n\
      Your task is to develop a Python function named \
      'calculate(df)'. This function should accept a dataframe as \
      its parameter. Ensure that you only utilize the columns \
      present in the dataset, specifically {df.columns}.\
      After processing, the function should return the processed \
      dataframe. Your response should strictly contain the Python \
      code for the 'calculate(df)' function \
      and exclude any unrelated content, do not import backtrader"
  }]

  reply_data = get_reply(msg)
  return reply_data

# 產生技術指標策略
def ai_strategy(df, user_msg, add_msg="無"):

  code_example ='''
class AiStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    if crossover(self.data.short_ma, self.data.long_ma):
        self.buy(size=1,
            sl=self.data.Close[-1] * 0.90,
            tp=self.data.Close[-1] * 1.10)
    elif crossover(self.data.long_ma, self.data.short_ma):
        self.sell(size=1,
             sl=self.data.Close[-1] * 1.10,
             tp=self.data.Close[-1] * 0.90)
        '''

  msg = [{
    "role":
    "system",
    "content":
     "As a Python code generation bot, your task is to generate \
     code for a stock strategy based on user requirements and df. \
     Please note that your response should solely \
     consist of the code itself, \
     and no additional information should be included."
  }, {
    "role":
    "user",
    "content":
     "The user requirement:計算 ma,\n\
     The additional requirement: 請設置 10% 的停利與停損點\n\
     The df.columns =['Open',	'High', 'Low',	'Close',	'Adj Close',	'Volume', 'short_ma',	'long_ma']\n\
     Please using the crossover() function in next(self)\
     Your response should strictly contain the Python \
     code for the 'AiStrategy(Strategy)' class \
     and exclude any unrelated content, don not import backtrader."
  }, {
    "role":
    "assistant",
    "content":f"{code_example}"
  }, {
    "role":
    "user",
    "content":
    f"The user requirement:{user_msg}\n\
     The additional requirement:{add_msg}\n\
     The df.columns ={df.columns}\n\
     Your task is to develop a Python class named \
     'AiStrategy(Strategy)'\
     Please using the crossover() function in next(self), do not import backtrader"

  }]

  reply_data = get_reply(msg)
  return reply_data


### 9️⃣ 計算技術指標

In [10]:
# 輸入股票代號
stock_id = "2330.tw"
# 抓取 5 年資料
#df = yf.download(stock_id, period="5y")
df = yf.Ticker(stock_id).history(period="5y")
# 計算指標
user_msg = ["MACD", "請設置10%的停損點與20%的停利點"]
#user_msg = ["RSI", "請設置10%的停損點與20%的停利點"]
code_str = ai_helper(df, user_msg[0])
code_str=code_str.replace('```','')
code_str=code_str.replace('python','')
print(code_str)
exec(code_str)
new_df = calculate(df)
new_df.tail()


import pandas as pd

def calculate(df):
    """
    Calculates the Moving Average Convergence Divergence (MACD) for the given DataFrame.

    Parameters:
    df (pd.DataFrame): The input DataFrame with at least a 'Close' column.

    Returns:
    pd.DataFrame: The DataFrame with 'MACD_12_26_9', 'MACD_Signal_12_26_9',
                  and 'MACD_Hist_12_26_9' columns added.
    """
    # Calculate the 12-period Exponential Moving Average (EMA) of the 'Close' price
    df['EMA_12'] = df['Close'].ewm(span=12, adjust=False).mean()

    # Calculate the 26-period Exponential Moving Average (EMA) of the 'Close' price
    df['EMA_26'] = df['Close'].ewm(span=26, adjust=False).mean()

    # Calculate the MACD Line
    df['MACD_12_26_9'] = df['EMA_12'] - df['EMA_26']

    # Calculate the 9-period EMA of the MACD Line (Signal Line)
    df['MACD_Signal_12_26_9'] = df['MACD_12_26_9'].ewm(span=9, adjust=False).mean()

    # Calculate the MACD Histogram
    df['MACD_Hist_12_26_9'] = df['MACD_12_26_9'

,Open,High,Low,Close,Volume,Dividends,Stock Splits,MACD_12_26_9,MACD_Signal_12_26_9,MACD_Hist_12_26_9
Date,,,,,,,,,,
2025-10-13 00:00:00+08:00,1390.0,1420.0,1390.0,1415.0,53850992,0.0,0.0,58.932698,50.599089,8.333609
2025-10-14 00:00:00+08:00,1455.0,1460.0,1420.0,1425.0,39889256,0.0,0.0,58.900610,52.259393,6.641217
2025-10-15 00:00:00+08:00,1435.0,1465.0,1425.0,1465.0,41056405,0.0,0.0,61.395122,54.086539,7.308583
2025-10-16 00:00:00+08:00,1465.0,1495.0,1465.0,1485.0,37737678,0.0,0.0,64.245293,56.118290,8.127003
2025-10-17 00:00:00+08:00,1455.0,1465.0,1450.0,1450.0,38077331,0.0,0.0,62.954170,57.485466,5.468704


### 🔟 策略生成

In [11]:
strategy_str = ai_strategy(new_df, user_msg[0], user_msg[1])
strategy_str=strategy_str.replace('```','')
strategy_str=strategy_str.replace('python','')
print(strategy_str)
print("-----------------------")
exec(strategy_str)
backtest = Backtest(df,
        AiStrategy,
        cash=100000,
        commission=0.004,
        trade_on_close=True,
        exclusive_orders=True,
        )
stats = backtest.run()
print(stats)

class AiStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    # MACD line crosses above Signal line (buy signal)
    if crossover(self.data.MACD_12_26_9, self.data.MACD_Signal_12_26_9):
        # Buy with 10% stop loss and 20% take profit
        self.buy(size=1,
            sl=self.data.Close[-1] * 0.90,
            tp=self.data.Close[-1] * 1.20)
    # MACD line crosses below Signal line (sell signal)
    elif crossover(self.data.MACD_Signal_12_26_9, self.data.MACD_12_26_9):
        # Sell with 10% stop loss and 20% take profit
        self.sell(size=1,
             sl=self.data.Close[-1] * 1.10,
             tp=self.data.Close[-1] * 0.80)
-----------------------


Backtest.run:   0%|          | 0/1214 [00:00<?, ?bar/s]

Start                     2020-10-19 00:00...
End                       2025-10-17 00:00...
Duration                   1824 days 00:00:00
Exposure Time [%]                    95.72016
Equity Final [$]                  99872.56959
Equity Peak [$]                  100287.24737
Commissions [$]                      550.3142
Return [%]                           -0.12743
Buy & Hold Return [%]               248.39261
Return (Ann.) [%]                    -0.02644
Volatility (Ann.) [%]                 0.21073
CAGR [%]                             -0.01762
Sharpe Ratio                         -0.12548
Sortino Ratio                        -0.17804
Calmar Ratio                         -0.04154
Alpha [%]                              0.1591
Beta                                 -0.00115
Max. Drawdown [%]                    -0.63659
Avg. Drawdown [%]                    -0.07782
Max. Drawdown Duration      731 days 00:00:00
Avg. Drawdown Duration       93 days 00:00:00
# Trades                          

### 1️⃣1️⃣ 寫成函式

In [12]:
def ai_backtest(stock_id, period, user_msg, add_msg):

  # 下載資料
  #df = yf.download(stock_id, period=period)
  df = yf.Ticker(stock_id).history(period="5y")

  # 獲取和執行指標計算程式碼
  code_str = ai_helper(df, user_msg)
  code_str=code_str.replace('```','')
  code_str=code_str.replace('python','')
  local_namespace = {}
  exec(code_str, globals(), local_namespace)
  calculate = local_namespace['calculate']
  new_df = calculate(df)

  # 獲取和執行策略程式碼
  strategy_str = ai_strategy(new_df, user_msg, add_msg)
  strategy_str=strategy_str.replace('```','')
  strategy_str=strategy_str.replace('python','')
  print(strategy_str)

  print("-----------------------")
  exec(strategy_str, globals(), local_namespace)
  AiStrategy = local_namespace['AiStrategy']

  backtest = Backtest(df,
          AiStrategy,
          cash=100000,
          commission=0.004,
          trade_on_close=True,
          exclusive_orders=True,
          )
  stats = backtest.run()
  print(stats)
  return str(stats)


## 5-4 讓 AI 解析回測報告

### 1️⃣2️⃣ 設定 AI 回復內容

In [13]:
def backtest_analysis(*args):

  content_list = [f"策略{i+1}：{report}"
                  for i, report in enumerate(args)]
  content = "\n".join(content_list)
  content += "\n\n請依資料給我一份約200字的分析報告。若有多個策略, \
                  請選出最好的策略及原因, reply in 繁體中文."

  msg = [{
      "role": "system",
      "content": "你是一位專業的證券分析師, 我會給你交易策略的回測績效,\
                  請幫我進行績效分析.不用詳細講解每個欄位, \
                  重點說明即可, 並回答交易策略的好壞"
  }, {
      "role": "user",
      "content": content
  }]

  reply_data = get_reply(msg)
  return reply_data


### 1️⃣3️⃣ 回測結果分析

In [14]:
stats = ai_backtest(stock_id="2330.TW",
           period="5y",
           user_msg="MACD",
           add_msg="請設置10%的停損點與20%的停利點")
reply = backtest_analysis(stats)
print(reply)


class AiStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    if crossover(self.data.MACD, self.data.Signal_Line):
        self.buy(size=1,
            sl=self.data.Close[-1] * 0.90,
            tp=self.data.Close[-1] * 1.20)
    elif crossover(self.data.Signal_Line, self.data.MACD):
        self.sell(size=1,
             sl=self.data.Close[-1] * 1.10,
             tp=self.data.Close[-1] * 0.80)
-----------------------


Backtest.run:   0%|          | 0/1214 [00:00<?, ?bar/s]

Start                     2020-10-19 00:00...
End                       2025-10-17 00:00...
Duration                   1824 days 00:00:00
Exposure Time [%]                    95.72016
Equity Final [$]                  99872.57207
Equity Peak [$]                  100287.24949
Commissions [$]                     550.31419
Return [%]                           -0.12743
Buy & Hold Return [%]               248.39256
Return (Ann.) [%]                    -0.02644
Volatility (Ann.) [%]                 0.21073
CAGR [%]                             -0.01761
Sharpe Ratio                         -0.12548
Sortino Ratio                        -0.17804
Calmar Ratio                         -0.04154
Alpha [%]                              0.1591
Beta                                 -0.00115
Max. Drawdown [%]                    -0.63659
Avg. Drawdown [%]                    -0.07782
Max. Drawdown Duration      731 days 00:00:00
Avg. Drawdown Duration       93 days 00:00:00
# Trades                          

### 1️⃣4️⃣ 比較多個策略

In [15]:
# 策略1:MACD+停利停損
stats1 = ai_backtest(stock_id="2330.TW", period="5y",
            user_msg="MACD",
            add_msg="請設置10%的停損點與20%的停利點")
# 策略2:SMA
stats2 = ai_backtest(stock_id="2330.TW", period="5y",
            user_msg="SMA",
            add_msg="無")
# 策略3:RSI+停利停損
stats3 = ai_backtest(stock_id="2330.TW", period="5y",
            user_msg="RSI",
            add_msg="請設置10%的停損點與20%的停利點")

reply = backtest_analysis(stats1, stats2, stats3)
print(reply)

class AiStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    if crossover(self.data.MACD, self.data.Signal_Line):
        self.buy(size=1,
            sl=self.data.Close[-1] * 0.90,
            tp=self.data.Close[-1] * 1.20)
    elif crossover(self.data.Signal_Line, self.data.MACD):
        self.sell(size=1,
             sl=self.data.Close[-1] * 1.10,
             tp=self.data.Close[-1] * 0.80)
-----------------------


Backtest.run:   0%|          | 0/1214 [00:00<?, ?bar/s]

Start                     2020-10-19 00:00...
End                       2025-10-17 00:00...
Duration                   1824 days 00:00:00
Exposure Time [%]                    95.72016
Equity Final [$]                  99872.56952
Equity Peak [$]                  100287.24705
Commissions [$]                     550.31419
Return [%]                           -0.12743
Buy & Hold Return [%]               248.39256
Return (Ann.) [%]                    -0.02644
Volatility (Ann.) [%]                 0.21073
CAGR [%]                             -0.01762
Sharpe Ratio                         -0.12548
Sortino Ratio                        -0.17804
Calmar Ratio                         -0.04154
Alpha [%]                              0.1591
Beta                                 -0.00115
Max. Drawdown [%]                    -0.63659
Avg. Drawdown [%]                    -0.07782
Max. Drawdown Duration      731 days 00:00:00
Avg. Drawdown Duration       93 days 00:00:00
# Trades                          

Backtest.run:   0%|          | 0/1214 [00:00<?, ?bar/s]

Start                     2020-10-19 00:00...
End                       2025-10-17 00:00...
Duration                   1824 days 00:00:00
Exposure Time [%]                    97.53086
Equity Final [$]                  99957.14632
Equity Peak [$]                  100196.74195
Commissions [$]                     668.82408
Return [%]                           -0.04285
Buy & Hold Return [%]               248.39256
Return (Ann.) [%]                    -0.00889
Volatility (Ann.) [%]                 0.22165
CAGR [%]                             -0.00592
Sharpe Ratio                         -0.04011
Sortino Ratio                        -0.05707
Calmar Ratio                         -0.01425
Alpha [%]                            -0.09096
Beta                                  0.00019
Max. Drawdown [%]                    -0.62372
Avg. Drawdown [%]                    -0.09931
Max. Drawdown Duration     1730 days 00:00:00
Avg. Drawdown Duration      257 days 00:00:00
# Trades                          

...

class AiStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    # Buy when RSI crosses above 30 (oversold)
    if crossover(self.data.RSI, 30):
        self.buy(size=1,
            sl=self.data.Close[-1] * 0.90,  # 10% stop-loss
            tp=self.data.Close[-1] * 1.20)  # 20% take-profit
    # Sell when RSI crosses below 70 (overbought)
    elif crossover(70, self.data.RSI):
        self.sell(size=1,
             sl=self.data.Close[-1] * 1.10,  # 10% stop-loss for short
             tp=self.data.Close[-1] * 0.80)  # 20% take-profit for short
-----------------------


Backtest.run:   0%|          | 0/1214 [00:00<?, ?bar/s]

Start                     2020-10-19 00:00...
End                       2025-10-17 00:00...
Duration                   1824 days 00:00:00
Exposure Time [%]                    86.66667
Equity Final [$]                  99089.48583
Equity Peak [$]                  100211.30939
Commissions [$]                     226.51503
Return [%]                           -0.91051
Buy & Hold Return [%]               248.39259
Return (Ann.) [%]                    -0.18953
Volatility (Ann.) [%]                 0.18374
CAGR [%]                             -0.12629
Sharpe Ratio                         -1.03154
Sortino Ratio                        -1.36691
Calmar Ratio                         -0.16418
Alpha [%]                            -0.43464
Beta                                 -0.00192
Max. Drawdown [%]                    -1.15438
Avg. Drawdown [%]                    -0.09437
Max. Drawdown Duration     1319 days 00:00:00
Avg. Drawdown Duration      111 days 00:00:00
# Trades                          

/tmp/ipython-input-897687173.py:33: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  stats = backtest.run()


根據您提供的兩份回測績效報告，以下是詳細分析：

**整體表現評估：**
兩項策略在回測期間（2020-10-19 至 2025-10-17）的表現都非常不理想。市場在同期實現了高達248.39%的買入並持有報酬，但兩個策略都呈現負報酬率，且遠遠落後於大盤。這表示策略未能有效捕捉市場上漲的機會，反而造成了虧損。

**策略1績效分析：**
*   **報酬：** 總報酬率-0.127%，年化報酬率-0.026%，年化複合成長率(CAGR)為-0.017%。顯示策略整體虧損。
*   **風險：** 最大回撤-0.636%相對較小，但夏普比率、索提諾比率及卡瑪比率均為負值，表示風險調整後的報酬表現差。
*   **交易統計：** 總交易次數97次，獲勝率31.96%。值得注意的是，損益因子(Profit Factor)為1.002，微幅大於1，且預期報酬率(Expectancy)為0.005%，表示平均每筆交易理論上能帶來極小的正收益，儘管整體策略仍虧損。

**策略2績效分析：**
*   **報酬：** 總報酬率-0.042%，年化報酬率-0.008%，年化複合成長率(CAGR)為-0.005%。雖然虧損幅度略小於策略1，但仍是負報酬。
*   **風險：** 最大回撤-0.623%與策略1相近，但最大回撤持續時間長達1730天，顯示資產處於虧損狀態的時間非常久。所有風險調整後報酬率指標均為負。
*   **交易統計：** 總交易次數126次，獲勝率僅18.25%，明顯低於策略1。損益因子(0.930)低於1，預期報酬率為負(-0.130%)，這表示平均每筆交易是虧損的。

**策略好壞及選擇：**

**這兩個策略都屬於表現不佳的策略。** 兩者都未能提供正向報酬，並且在市場大幅上漲的背景下，大幅跑輸簡單的買入並持有策略。

如果必須從中選出一個「相對較好」的策略，我會選擇**策略1**。

**原因如下：**
儘管策略1的整體報酬也是負值，但其在交易層面的指標表現優於策略2：
1.  **損益因子 (Profit Factor)：** 策略1為1.002，略高於1，而策略2為0.930，低於1。這表示策略1的總盈利略大於總虧損，而策略2的總虧損大於總盈利。
2.  **預期報酬率 (Expectancy)：** 策略1為0.005%（正值），策略2為-0.13

## 5-5 策略之回測與繪圖

In [16]:
# 60MA 高於季線做多，跌破出場

class OneMA(Strategy):
    n1 = 60  #預設的均線參數

    def init(self): #初始化會用到的參數和指標，告知要如何計算
        self.sma1 = self.I(SMA, self.data.Close, self.n1)

    def next(self): #回測的時候每一根K棒出現什麼狀況要觸發進出場
        #如果收盤價>sma1(也就是60ma)，而且目前沒有多單部位
        if (self.data.Close > self.sma1) and (not self.position.is_long) :
            self.buy()#做多
        #如果收盤價<sma1(也就是60ma)
        elif (self.data.Close < self.sma1):
            self.position.close()#部位出場
                                 #如果要做空就用self.sell()

In [17]:
backtest = Backtest(df,
        OneMA,
        cash=100000,
        commission=0.004,
        margin=1,
        hedging=False,
        trade_on_close=False,
        exclusive_orders=False,
        )
stats = backtest.run()

# 印出回測績效
print(stats)

# 查看詳細的交易紀錄
stats["_trades"].head()

Backtest.run:   0%|          | 0/1155 [00:00<?, ?bar/s]

Start                     2020-10-19 00:00...
End                       2025-10-17 00:00...
Duration                   1824 days 00:00:00
Exposure Time [%]                    56.87243
Equity Final [$]                 130936.57894
Equity Peak [$]                  134086.57894
Commissions [$]                   22062.67696
Return [%]                           30.93658
Buy & Hold Return [%]               171.59481
Return (Ann.) [%]                     5.74974
Volatility (Ann.) [%]                22.93459
CAGR [%]                              3.79416
Sharpe Ratio                           0.2507
Sortino Ratio                         0.40274
Calmar Ratio                          0.11356
Alpha [%]                           -60.72436
Beta                                  0.53417
Max. Drawdown [%]                   -50.63332
Avg. Drawdown [%]                   -14.78826
Max. Drawdown Duration     1264 days 00:00:00
Avg. Drawdown Duration      288 days 00:00:00
# Trades                          

/tmp/ipython-input-2353848551.py:10: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  stats = backtest.run()


,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,SL,TP,PnL,Commission,ReturnPct,EntryTime,ExitTime,Duration,Tag,"Entry_SMA(C,60)","Exit_SMA(C,60)"
0,183,61,104,543.939639,525.054749,None,None,-4238.438797,782.503892,-0.042580,2021-01-13 00:00:00+08:00,2021-03-25 00:00:00+08:00,71 days,None,452.193250,544.410991
1,173,107,108,550.756652,547.085144,None,None,-1394.877330,759.706523,-0.014640,2021-03-30 00:00:00+08:00,2021-03-31 00:00:00+08:00,1 days,None,548.361997,549.571826
2,166,110,114,563.607870,554.428568,None,None,-2266.140202,742.376195,-0.024222,2021-04-07 00:00:00+08:00,2021-04-13 00:00:00+08:00,6 days,None,552.481423,557.615539
3,163,116,118,561.771892,555.346537,None,None,-1775.694036,728.361216,-0.019392,2021-04-15 00:00:00+08:00,2021-04-19 00:00:00+08:00,4 days,None,559.825440,560.937202
4,163,148,153,548.920959,541.577596,None,None,-1907.973288,711.005058,-0.021324,2021-06-01 00:00:00+08:00,2021-06-08 00:00:00+08:00,7 days,None,543.532975,543.337190


In [18]:
backtest.plot(plot_equity=True,
       plot_return=False,
       plot_pl=True,
       plot_volume=True,
       plot_drawdown=False,
       superimpose=True)

/usr/local/lib/python3.12/dist-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


GridPlot(id='p1693', ...)

## References:

### [Backtesting - 均線突破策略](https://ithelp.ithome.com.tw/articles/10274546)

### [Backtesting - 參數最佳化](https://ithelp.ithome.com.tw/articles/10274547)